In [1]:
include("../src/Julia.jl")

parseData (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
gpath = "../../graphs/";

In [4]:
graphs = readdir("../../graphs");
graphs = graphs[1:ceil(Int64,length(graphs)/4)]

7-element Array{ByteString,1}:
 "grid2_10000_iu1000"   
 "grid2_10000_iu11"     
 "grid2_10000_u1000"    
 "grid3_27000_iu1000"   
 "grid3_27000_u1000"    
 "grid3_9261_iu11"      
 "ipmOri_grid3_125000_1"

In [5]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
        x = readFromFile(path * "x.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        x = x - mean(x)
        b = la * x; b = b - mean(b);
        
        numIts = 1000;
        @time for i in 1:5
            @time F = treeSolver(tree[i])
            @time F(b);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - b) / norm(b)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now useing higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                xHP = toHighPrecision(x; precision=hp);
                bHP = toHighPrecision(b; precision=hp);
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur, "\n")
    catch
        println("skipping ", cur, "\n")
    end
end

  0.526795 seconds (652.03 k allocations: 30.342 MB, 1.71% gc time)
  0.164421 seconds (290.91 k allocations: 7.538 MB)
 10.210310 seconds (198.37 M allocations: 4.678 GB, 5.80% gc time)
497.399088 seconds (2.38 G allocations: 60.478 GB, 33.86% gc time)
561.148821 seconds (2.38 G allocations: 66.258 GB, 34.55% gc time)
591.751263 seconds (2.38 G allocations: 77.836 GB, 33.65% gc time)
671.189128 seconds (2.38 G allocations: 100.992 GB, 33.04% gc time)
782.124297 seconds (2.38 G allocations: 147.303 GB, 32.25% gc time)
  0.028579 seconds (50.18 k allocations: 5.761 MB)
  0.049919 seconds (197.89 k allocations: 3.631 MB, 78.92% gc time)
 13.827058 seconds (198.16 M allocations: 4.669 GB, 20.55% gc time)
505.700553 seconds (2.38 G allocations: 60.469 GB, 33.93% gc time)
566.225321 seconds (2.38 G allocations: 66.258 GB, 34.32% gc time)
654.239947 seconds (2.38 G allocations: 77.836 GB, 32.93% gc time)
733.352665 seconds (2.38 G allocations: 100.992 GB, 32.79% gc time)
840.471590 seconds (